# setup

In [1]:
!pip install openai tiktoken -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 328.8/328.8 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 12.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.1 MB/s eta 0:00:00


In [2]:
import openai
import tiktoken
from google.colab import userdata

api_key = userdata.get('OPENAI_API_KEY')

client = openai.Client(api_key=api_key)

# helper function

In [12]:
def get_completion(prompt, model='gpt-3.5-turbo'):
  messages = [{'role': 'user', 'content': prompt}]
  response = client.chat.completions.create(
      model=model,
      messages=messages,
      temperature=1
  )

  return response.choices[0].message.content

# helper function in chat format

In [13]:
def get_completion_from_messages(messages, model='gpt-3.5-turbo', temperature=0, max_tokens=500):
  response = client.chat.completions.create(
      model=model,
      messages=messages,
      temperature=temperature, # degree of randomness
      max_tokens=max_tokens # max number of tokens that model can output
  )

  return response.choices[0].message.content

# Moderating output for potentially harmful content

In [3]:
final_response_to_customer = f"""
The SmartX ProPhone has a 6.1-inch display, 128GB storage, \
12MP dual camera, and 5G. The FotoSnap DSLR Camera \
has a 24.2MP sensor, 1080p video, 3-inch LCD, and \
interchangeable lenses. We have a variety of TVs, including \
the CineView 4K TV with a 55-inch display, 4K resolution, \
HDR, and smart TV features. We also have the SoundMax \
Home Theater system with 5.1 channel, 1000W output, wireless \
subwoofer, and Bluetooth. Do you have any specific questions \
about these products or any other products we offer?
"""

response = client.moderations.create(
    input=final_response_to_customer
)

response

ModerationCreateResponse(id='modr-9nFK4GFLxTAme31yrQmVMm8OkU5p3', model='text-moderation-007', results=[Moderation(categories=Categories(harassment=False, harassment_threatening=False, hate=False, hate_threatening=False, self_harm=False, self_harm_instructions=False, self_harm_intent=False, sexual=False, sexual_minors=False, violence=False, violence_graphic=False, self-harm=False, sexual/minors=False, hate/threatening=False, violence/graphic=False, self-harm/intent=False, self-harm/instructions=False, harassment/threatening=False), category_scores=CategoryScores(harassment=2.696166302484926e-05, harassment_threatening=9.87596831691917e-06, hate=7.229043148981873e-06, hate_threatening=2.0055701952514937e-06, self_harm=1.2812188288080506e-06, self_harm_instructions=3.672591049053153e-07, self_harm_intent=2.012526920225355e-06, sexual=0.00015211118443403393, sexual_minors=1.154503297584597e-05, violence=0.0002972284273710102, violence_graphic=1.5082588106452022e-05, self-harm=1.2812188288

In [7]:
moderation_output = response.results[0].flagged
print(moderation_output)

False


## check if output is based on the product provided information

In [14]:
system_message = f"""
you are an assitant that evaluates whether \
customer service agent responses sufficiently \
answer customer questions, and also validates that \
all the facts the assistant cites from the product \
information are correct.

the product information and user and customer \
service agent messages will be delimited by \
3 backticks, i.e. ```.

Respond with a Y or N character, with no punctuation:
Y - if the output sufficiently answers the question \
and the response correctly uses the product information
N - otherwise

Output a single letter only
"""

customer_message = f"""
tell me about the smartx prophone and \
the fotosnap camera, the dslr one. \
Also tell me about your tvs.
"""

product_information = """{ "name": "SmartX ProPhone", "category": "Smartphones and Accessories", "brand": "SmartX", "model_number": "SX-PP10", "warranty": "1 year", "rating": 4.6, "features": [ "6.1-inch display", "128GB storage", "12MP dual camera", "5G" ], "description": "A powerful smartphone with advanced camera features.", "price": 899.99 } { "name": "FotoSnap DSLR Camera", "category": "Cameras and Camcorders", "brand": "FotoSnap", "model_number": "FS-DSLR200", "warranty": "1 year", "rating": 4.7, "features": [ "24.2MP sensor", "1080p video", "3-inch LCD", "Interchangeable lenses" ], "description": "Capture stunning photos and videos with this versatile DSLR camera.", "price": 599.99 } { "name": "CineView 4K TV", "category": "Televisions and Home Theater Systems", "brand": "CineView", "model_number": "CV-4K55", "warranty": "2 years", "rating": 4.8, "features": [ "55-inch display", "4K resolution", "HDR", "Smart TV" ], "description": "A stunning 4K TV with vibrant colors and smart features.", "price": 599.99 } { "name": "SoundMax Home Theater", "category": "Televisions and Home Theater Systems", "brand": "SoundMax", "model_number": "SM-HT100", "warranty": "1 year", "rating": 4.4, "features": [ "5.1 channel", "1000W output", "Wireless subwoofer", "Bluetooth" ], "description": "A powerful home theater system for an immersive audio experience.", "price": 399.99 } { "name": "CineView 8K TV", "category": "Televisions and Home Theater Systems", "brand": "CineView", "model_number": "CV-8K65", "warranty": "2 years", "rating": 4.9, "features": [ "65-inch display", "8K resolution", "HDR", "Smart TV" ], "description": "Experience the future of television with this stunning 8K TV.", "price": 2999.99 } { "name": "SoundMax Soundbar", "category": "Televisions and Home Theater Systems", "brand": "SoundMax", "model_number": "SM-SB50", "warranty": "1 year", "rating": 4.3, "features": [ "2.1 channel", "300W output", "Wireless subwoofer", "Bluetooth" ], "description": "Upgrade your TV's audio with this sleek and powerful soundbar.", "price": 199.99 } { "name": "CineView OLED TV", "category": "Televisions and Home Theater Systems", "brand": "CineView", "model_number": "CV-OLED55", "warranty": "2 years", "rating": 4.7, "features": [ "55-inch display", "4K resolution", "HDR", "Smart TV" ], "description": "Experience true blacks and vibrant colors with this OLED TV.", "price": 1499.99 }"""

q_a_pair = f"""
customer message: ```{customer_message}```
product_information: ```{product_information}```
agent_response: ```{final_response_to_customer}```

Does the response use the retrieved information correctly?
Does the response sufficiently answer the question?

Output Y or N
"""

messages = [
    dict(role='system', content=system_message),
    dict(role='user', content=q_a_pair)
]

response = get_completion_from_messages(messages)
print(response)

Y


In [15]:
another_response = "life is like a box of chocolates"
q_a_pair = f"""
Customer message: ```{customer_message}```
Product information: ```{product_information}```
Agent response: ```{another_response}```

Does the response use the retrieved information correctly?
Does the response sufficiently answer the question?

Output Y or N
"""
messages = [
    {'role': 'system', 'content': system_message},
    {'role': 'user', 'content': q_a_pair}
]

response = get_completion_from_messages(messages)
print(response)

N
